In [ ]:
'''
Moedl Used - EB0 and Xception
Full trainnig of twin backbone
'''

In [ ]:
!pip install tensorflow-addons==0.9.1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, concatenate
from keras.engine import Layer, InputSpec
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K
from keras.activations import elu
from keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Metric
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow.keras.applications import ResNet50, ResNet101, DenseNet121, InceptionResNetV2, Xception
from tensorflow.keras.applications import InceptionV3,ResNet152V2,ResNet50V2
import os
from tqdm import tqdm
import seaborn as sns

In [ ]:
!pip install efficientnet
import efficientnet.tfkeras as efn

In [ ]:
train_df = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
print(train_df.shape)
train_df.head()

In [ ]:
!ls ../input

In [ ]:
train_images_path = '../input/panda-cancer-l2-sqtile32-v1/train_squaretile_images/'

In [ ]:
img = plt.imread(train_images_path + train_df.loc[0]['image_id'] + '.png')
plt.imshow(img)
print(img.shape)

In [ ]:
train_df["image_path"] = train_df["image_id"].apply(lambda x: x + '.png')

Creating DataFrame for only available images in train directory

In [ ]:
new_img_df = pd.DataFrame()

In [ ]:
train_tile_images = list(os.listdir(train_images_path))

In [ ]:
len(train_tile_images)

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    if train_df['image_path'][i] in train_tile_images:
        new_img_df = new_img_df.append(train_df.loc[i])

In [ ]:
new_img_df

In [ ]:
sns.countplot(new_img_df['isup_grade'])

In [ ]:
new_img_df['isup_grade'][:7000].value_counts()

In [ ]:
new_img_df[new_img_df['isup_grade'] == 0].sample(frac=0.451)

In [ ]:
new_img_df[new_img_df['isup_grade'] == 1].sample(frac=0.488)

In [ ]:
new_train_df = new_img_df[(new_img_df['isup_grade'] != 0) & (new_img_df['isup_grade'] != 1)]

In [ ]:
new_train_df

In [ ]:
new_train_df = new_train_df.append(new_img_df[new_img_df['isup_grade'] == 0].sample(frac=0.451))

In [ ]:
new_train_df = new_train_df.append(new_img_df[new_img_df['isup_grade'] == 1].sample(frac=0.488))

In [ ]:
new_train_df = new_train_df.sample(frac=1)

In [ ]:
new_train_df.reset_index(inplace = True) 

In [ ]:
new_train_df = new_train_df.drop(['index'],axis=1)

In [ ]:
new_train_df

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(new_train_df["image_path"], new_train_df["isup_grade"], test_size = 0.15, stratify = new_train_df["isup_grade"])

df_train = pd.DataFrame({"image_path":xtrain, "isup_grade":ytrain})
df_val = pd.DataFrame({"image_path":xval, "isup_grade":yval})

df_train["isup_grade"] = df_train["isup_grade"].astype('str')
df_val["isup_grade"] = df_val["isup_grade"].astype('str')

In [ ]:
print(df_train.shape) 
print(df_val.shape)

In [ ]:
BATCH_SIZE = 4
img_size = 768
EPOCHS = 16
nb_classes = 6

In [ ]:
LR_START = 0.00001
LR_MAX = 0.0001 * 8
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 3
LR_SUSTAIN_EPOCHS = 1
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
# Input layer to both backbone layer
input_layer = tf.keras.Input(shape=(img_size, img_size, 3))

def get_model():
    base_model1 =  ResNet152V2(weights='imagenet', include_top=False, pooling='avg', input_tensor = input_layer,
                            input_shape=(img_size, img_size, 3))
    # renaming all layer
    for i, layer in enumerate(base_model1.layers):
        layer._name = 'm1_' + str(i)
    
    x1 = base_model1.output
    
    #base_model2 =  ResNet152V2(weights='imagenet', include_top=False, pooling='avg', input_tensor = input_layer,
     #                        input_shape=(img_size, img_size, 3))
   
    
    base_model2 =  efn.EfficientNetB0(weights='imagenet', include_top=False, pooling='avg', input_tensor = input_layer,
                             input_shape=(img_size, img_size, 3))
    # renaming all layer
    for i, layer in enumerate(base_model2.layers):
        layer._name = 'm2_' + str(i)

    x2 = base_model2.output
           
    x = concatenate([x1,x2])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs = input_layer, outputs=predictions)

In [ ]:
model = get_model()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

# Image Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
#         horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

valid_datagen = ImageDataGenerator(
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
#         horizontal_flip=True,  # randomly flip images
        vertical_flip=True)   # randomly flip images

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = df_train,
                                               directory = train_images_path,
                                               x_col = "image_path",
                                               y_col = "isup_grade",
                                               batch_size = BATCH_SIZE,
                                               target_size =  (img_size, img_size),
                                               class_mode = 'categorical')

validation_generator = valid_datagen.flow_from_dataframe(dataframe = df_val,
                                                    directory = train_images_path,
                                                    x_col = "image_path",
                                                    y_col = "isup_grade",
                                                    batch_size = BATCH_SIZE, 
                                                    target_size = (img_size, img_size),
                                                    class_mode = 'categorical')

In [ ]:
# checkpoint
filepath="enemble-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
# %%time
history = model.fit_generator(
            generator = train_generator,
            steps_per_epoch = (df_train.shape[0] // BATCH_SIZE),
            epochs=EPOCHS,
            validation_data = validation_generator , 
            validation_steps = (df_val.shape[0] // BATCH_SIZE),
            callbacks=[lr_callback, checkpoint_callback]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

# cohens_kappa = history.history['cohen_kappa']
# val_cohens_kappa = history.history['val_cohen_kappa']

epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()
 
# plt.plot(epochs, cohens_kappa, 'b', label='Training Cohen-kappa')
# plt.plot(epochs, val_cohens_kappa, 'r', label='Validation Cohen-kappa')
# plt.title('Cohen Kappa - Training and validation score')
# plt.legend()

plt.show()

In [ ]:
#model.save('ensemble_model_7k_data.h5')